In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
from scipy.cluster.hierarchy import linkage

import extq
import pyemma
import ivac

import glob
import sys
import importlib
import gc

import sklearn
from sklearn import metrics

/project2/roux/scguo/anaconda/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/project2/roux/scguo/anaconda/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/project2/roux/scguo/anaconda/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/project2/roux/scguo/anaconda/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/project2/roux/scguo/anaconda/envs/py37/lib/pyth

In [3]:
sys.path.insert(1, "../../python")
sys.path.insert(1, "../../..")
from util import *
import kde

In [4]:
plt.style.use("seaborn-ticks")
sns.set_palette("colorblind")

# Load data and CVs 

In [5]:
livac_trajs = np.load(
    "../../data/sb_ind100_TICA/livac_trajs_anton.npy", allow_pickle=True
)
ri_trajs = np.load("../../data/raw_feat/ri126_dist_anton.npz.npy", allow_pickle=True)
cv_trajs = np.load("../../data/raw_feat/cv_dist_spin_anton.npy", allow_pickle=True)

In [6]:
sb_labels = []
for r in ("R217", "R223", "R226", "R229", "R232"):
    for n in ("D129", "D136", "D151", "D164", "E183", "D186"):
        sb_labels.append(f"{r} - {n}")

In [7]:
remove = [
    1282,
    1283,
    1284,
    1285,
    1286,
    1288,
    1289,
    1290,
    1187,
    1188,
    1189,
    1190,
    1191,
    1197,
    1198,
    1199,
    1203,
    1205,
    1206,
    1207,
    1211,
    1212,
    1213,
    1214,
    1215,
    1225,
    1226,
    1227,
    1228,
    1231,
    1232,
    1233,
    1236,
    1237,
    1238,
    1242,
    1245,
    1246,
    1252,
    1253,
    1260,
    1261,
    1262,
    1263,
    1266,
    1267,
    1268,
    1269,
    1270,
    1271,
    1272,
    1273,
    1274,
    1275,
    1276,
    1277,
    1278,
    1279,
]

In [8]:
print(len(livac_trajs), len(ri_trajs), len(cv_trajs))

livac_arr = np.concatenate(livac_trajs)
ri_arr = np.concatenate(ri_trajs)
cv_arr = np.concatenate(cv_trajs)
print(livac_arr.shape, ri_arr.shape, cv_arr.shape)

237 237 237
(3000000, 5) (3000000, 5) (3000000, 2)


In [9]:
ivac_trajs = np.load("../../data/feat2_anton_ivac.npy", allow_pickle=True)
ivac_arr = np.concatenate(ivac_trajs)
print(len(ivac_trajs), ivac_trajs[0].shape)
print(ivac_arr.shape)

237 (10000, 10)
(3000000, 10)


In [10]:
sb_trajs = np.load("../../data/raw_feat/feat2_raw_anton.npy", allow_pickle=True)
sb_arr = np.concatenate(sb_trajs)
print(sb_trajs.shape)
print(sb_arr.shape)

(237,)
(3000000, 60)


## States 

In [11]:
dist = cv_arr[:, 0]
rot = cv_arr[:, 1]
tic1 = livac_arr[:, 1]
tic2 = livac_arr[:, 2]
sb1 = sb_arr[:, 42]  # R226-D129
sb2 = sb_arr[:, 48]  # R229-D129
sb3 = sb_arr[:, 53]  # R229-D186
sb4 = sb_arr[:, 54]  # R4 - D129

# ellipse centered at (-0.506, 3.94) with radii 0.84 nm and 7.6 deg (2 sigma in each direction) for up state
# take points where TIC 2 > -1.0 and R3/R4-D129 salt bridges < 0.5/> 0.6 nm, R3-D186 < 0.55
up_ids = (
    ((((dist + 0.506) / 0.84) ** 2 + ((rot - 3.94) / 7.6) ** 2) < 1)
    & (sb1 < 0.5)
    & (sb2 > 0.6)
    & (sb3 < 0.55)
    & (tic2 > -1)
).nonzero()[0]

# ellipse centered at (6.447, 60.713) with radii 0.632 and 2.9 for up+ state
# take points where TIC 1 < -1.25
dist_ax_upp = (dist - 6.447) / 0.632
rot_ax_upp = (rot - 60.713) / 4.0
upplus_ids = (
    ((dist_ax_upp ** 2 + rot_ax_upp ** 2) < 1) & (tic1 < -1.25) & (sb4 < 1.0)
).nonzero()[0]

In [12]:
mask = np.ones(len(cv_arr), dtype=bool)
mask[upplus_ids] = False
mask[up_ids] = False

# len(**_ids) is number of points in each state
# upplus = data_arr[upplus_ids]
# up = data_arr[up_ids]
# other = data_arr[mask]

# split CV data
upplus_cvs = cv_arr[upplus_ids]
up_cvs = cv_arr[up_ids]
other_cvs = cv_arr[mask]
upplus_sbs = sb_arr[upplus_ids]
up_sbs = sb_arr[up_ids]
other_sbs = cv_arr[mask]

In [13]:
assert (len(upplus_cvs) + len(up_cvs) + len(other_cvs)) == len(cv_arr)
print(f"Number of up+ structures:  {len(upplus_cvs)}")
print(f"Number of up structures:   {len(up_cvs)}")
print(f"Number of other structures: {len(other_cvs)}")

Number of up+ structures:  6946
Number of up structures:   3396
Number of other structures: 2989658


# Clustering

In [14]:
feat5_ivac_trajs = np.load(
    "../../data/raw_feat/feat5_ivac_anton.npy", allow_pickle=True
)

feat5_ivac_arr = np.concatenate(feat5_ivac_trajs)
print(feat5_ivac_arr.shape)

(3000000, 10)


In [15]:
from sklearn.cluster import AgglomerativeClustering

In [16]:
# use single-linkage hierarchical clustering
single = AgglomerativeClustering(n_clusters=1000, linkage="single").fit(feat5_ivac_arr)

In [19]:
dtrajs = single.labels_
print(dtrajs[:10])

[0 0 0 0 0 0 0 0 0 0]


In [22]:
single.n_clusters

1000

In [23]:
single.n_connected_components_

1

In [36]:
np.count_nonzero(dtrajs == 0)

2998959

In [32]:
# ward hierarchical clustering
# subsample by 20
ward = AgglomerativeClustering(n_clusters=1000, linkage="ward").fit(
    feat5_ivac_arr[::20]
)

MemoryError: unable to allocate array data.

In [37]:
# use average-linkage hierarchical clustering
avg = AgglomerativeClustering(n_clusters=1000, linkage="average").fit(
    feat5_ivac_arr[::10]
)

MemoryError: Unable to allocate 335. GiB for an array with shape (44999850000,) and data type float64